In [6]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

num_records = 100

# Metro Manila cities with accurate coordinates
metro_manila_coords = {
    "Manila": (14.5995, 120.9842),
    "Makati": (14.5547, 121.0244),
    "Quezon City": (14.6760, 121.0437),
    "Taguig": (14.5176, 121.0509),
    "Pasig": (14.5764, 121.0851),
    "San Juan": (14.6019, 121.0352),
    "Mandaluyong": (14.5794, 121.0359),
    "Parañaque": (14.4793, 121.0198),
    "Las Piñas": (14.4500, 120.9800),
    "Muntinlupa": (14.4081, 121.0415),
    "Pasay": (14.5378, 121.0014),
    "Caloocan": (14.6514, 120.9842),
    "Malabon": (14.6680, 120.9567),
    "Navotas": (14.6661, 120.9417),
    "Valenzuela": (14.7011, 120.9830),
    "Marikina": (14.6507, 121.1029),
    "Pateros": (14.5411, 121.0684)
}

# Step 1: Generate raw data with timestamps
raw_data = []

for _ in range(num_records):
    current_location = np.random.choice(list(metro_manila_coords.keys()))
    delivery_location = np.random.choice(list(metro_manila_coords.keys()))

    current_lat, current_lon = metro_manila_coords[current_location]
    delivery_lat, delivery_lon = metro_manila_coords[delivery_location]

    # Add slight randomization
    current_lat += np.random.uniform(-0.002, 0.002)
    current_lon += np.random.uniform(-0.002, 0.002)
    delivery_lat += np.random.uniform(-0.002, 0.002)
    delivery_lon += np.random.uniform(-0.002, 0.002)

    perishable = np.random.choice(["Yes", "No"])
    if perishable == "Yes":
        temperature = round(np.random.uniform(2.0, 12.0), 2)
        temperature_issue = "Yes - Temperature Exceeded Limit" if temperature > 7.0 else "Normal"
    else:
        temperature = round(np.random.uniform(10.0, 30.0), 2)
        temperature_issue = "N/A"

    order_date = datetime.now() - timedelta(days=np.random.randint(2, 5))
    expected_delivery_date = order_date + timedelta(days=np.random.randint(1, 3))

    # Determine status
    if round(current_lat, 3) == round(delivery_lat, 3) and round(current_lon, 3) == round(delivery_lon, 3):
        status = np.random.choice(["Delivered", "Awaiting Pickup"])
    else:
        status = np.random.choice(["In Transit", "Delayed"])

    # Timestamp = last updated
    timestamp = datetime.now() if status == "Delayed" else datetime.now() - timedelta(minutes=np.random.randint(30, 1440))

    raw_data.append({
        "timestamp": timestamp,
        "order_date": order_date,
        "expected_delivery_date": expected_delivery_date,
        "current_location": current_location,
        "delivery_location": delivery_location,
        "current_latitude": round(current_lat, 6),
        "current_longitude": round(current_lon, 6),
        "delivery_latitude": round(delivery_lat, 6),
        "delivery_longitude": round(delivery_lon, 6),
        "perishable": perishable,
        "temperature_celsius": temperature,
        "temperature_issue": temperature_issue,
        "status": status
    })

# Step 2: Sort by timestamp
sorted_data = sorted(raw_data, key=lambda x: x["timestamp"])

# Step 3: Assign package_id and rfid_tag based on order
for idx, record in enumerate(sorted_data, start=1):
    record["package_id"] = f"PKG{str(idx).zfill(3)}"
    record["rfid_tag"] = f"RFID{str(idx).zfill(3)}"
    record["timestamp"] = record["timestamp"].strftime("%Y-%m-%d %H:%M:%S")
    record["order_date"] = record["order_date"].strftime("%Y-%m-%d")
    record["expected_delivery_date"] = record["expected_delivery_date"].strftime("%Y-%m-%d")

# Step 4: Create DataFrame and save
df = pd.DataFrame(sorted_data)
df.to_csv("iot_data.csv", index=False)
df.to_json("iot_data.json", orient="records")

# Preview
df.head()

,timestamp,order_date,expected_delivery_date,current_location,delivery_location,current_latitude,current_longitude,delivery_latitude,delivery_longitude,perishable,temperature_celsius,temperature_issue,status,package_id,rfid_tag
0,2025-05-09 23:20:55,2025-05-07,2025-05-08,Parañaque,Pasig,14.479403,121.020347,14.575879,121.086845,Yes,11.31,Yes - Temperature Exceeded Limit,In Transit,PKG001,RFID001
1,2025-05-09 23:22:55,2025-05-07,2025-05-08,Pateros,Caloocan,14.539882,121.069672,14.650517,120.985020,No,26.88,N/A,In Transit,PKG002,RFID002
2,2025-05-10 00:14:55,2025-05-08,2025-05-10,Makati,Las Piñas,14.555964,121.025939,14.448395,120.978304,Yes,9.15,Yes - Temperature Exceeded Limit,In Transit,PKG003,RFID003
3,2025-05-10 00:53:55,2025-05-07,2025-05-09,Mandaluyong,Malabon,14.577454,121.037761,14.667079,120.956903,No,22.61,N/A,In Transit,PKG004,RFID004
4,2025-05-10 01:00:55,2025-05-08,2025-05-09,Taguig,Taguig,14.515796,121.052182,14.515960,121.049017,No,22.70,N/A,In Transit,PKG005,RFID005
